## Importamos Fenics

In [55]:
%%capture
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin


## Importamos la malla

In [56]:
%%bash
dolfin-convert malla.msh malla.xml
#dolfin-convert Actividad2_gmsh.msh Actividad2_gmsh.xml


Converting from Gmsh format (.msh, .gmsh) to DOLFIN XML format
Expecting 7424 vertices
Found all vertices
Expecting 33444 cells
Found all cells
Conversion done


In [57]:
from fenics import *

mesh = Mesh("malla.xml");
subdomains = MeshFunction('size_t',mesh,"malla_physical_region.xml");
boundary_markers = MeshFunction('size_t',mesh,"malla_facet_region.xml");

#mesh = Mesh("Actividad2_gmsh.xml");
#subdomains = MeshFunction('size_t',mesh,"Actividad2_gmsh_physical_region.xml");
#boundary_markers = MeshFunction('size_t',mesh,"Actividad2_gmsh_facet_region.xml");


## Desarrollanos el problema

`Definimos espacios de funciones`

In [58]:
# Definir el espacio de funciones para la temperatura (escalar)
P1 = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
VT = FunctionSpace(mesh, P1)

# Definir el espacio de funciones para el desplazamiento (vectorial, 3D)
P2 = VectorElement("Lagrange", mesh.ufl_cell(), 1)
Vu = FunctionSpace(mesh, P2)


`Definimos constantes y condiciones de borde`

In [59]:
# Parámetros físicos
mu = Constant(1.0)  # Módulo de elasticidad
kappa = Constant(1.0)  # Conductividad térmica

# Definir condiciones de borde
L = 10.0  # Longitud del extremo derecho (ajustar según la geometría)
T_criogenica = -100.0  # Temperatura criogénica
T_ambiente = 20.0  # Temperatura ambiente
torque_magnitud = 0.1  # Magnitud del torque aplicado

# Función para el extremo derecho (voladizo)
def right_end(x, on_boundary):
    return near(x[0], L) and on_boundary

def left_end(x, on_boundary):
    return (near(x[0], 0) or near(x[0], L)) and on_boundary

# Condiciones de temperatura
bc_T_criogenica = DirichletBC(VT, Constant(T_criogenica),boundary_markers, 10)
bc_T_ambiente = DirichletBC(VT, Constant(T_ambiente),boundary_markers, 20)

# Condiciones de desplazamiento
bcu = DirichletBC(Vu, Constant((0, 0, 0)), boundary_markers, 20)


`Definimos formulaciones variacionales`

In [60]:
# Variables simbólicas
dT = TrialFunction(VT)
T_ = TestFunction(VT)
du = TrialFunction(Vu)
u_ = TestFunction(Vu)
T_sol = Function(VT)

# Formulación variacional para temperatura (conducción de calor)
a_T = inner(grad(dT), kappa * grad(T_)) * dx
L_T = Constant(0) * dT * dx

# Variables elasticas
sigma = lambda u: 2.0 * mu * sym(grad(u))
epsilon = lambda u: sym(grad(u))

# Medida para la aplicación de condiciones en el contorno
ds = Measure('ds', domain=mesh, subdomain_data=boundary_markers)

# Arreglo vectorial del torque
e3 = Constant((0, 0, 1)) # Eje z unitario
r = as_vector((0, 1, 0)) # Vector radial
torque = torque_magnitud * cross(r, e3) # Torque como un momento: tau = r x F

# Formulación variacional para elasticidad lineal
a_M = inner(sigma(du), epsilon(u_)) * dx
L_M = inner(torque, u_) * ds(10)
#L_M = inner(Constant((0, 0, 0)), u_) * dx + inner(Constant((0, 0, T_aplicado)), u_) * ds(10)
#L_M = inner(Constant((0, 0)), u_) * dx + inner(Constant((0, T_aplicado)), u_) * ds


`Resolvemos los problemas y lo guardamos`

In [63]:
# Resolver problema de temperatura
A_T, b_T = assemble_system(a_T, L_T, bc_T_criogenica)
solve(A_T, T_sol.vector(), b_T)

# Resolver problema de elasticidad
A_M, b_M = assemble_system(a_M, L_M, bcu)
u_sol = Function(Vu)
solve(A_M, u_sol.vector(), b_M)

# Guardar solución de temperatura y desplazamiento en archivos
file_T = File("temperatura.pvd")
file_T << T_sol

file_u = File("desplazamiento.pvd")
file_u << u_sol
